# How AES Works

## Keyed Permutations

AES, como todos los buenos cifradores de bloques, realiza una "permutación con clave", es decir, asigna cada bloque de entrada posible a un bloque de salida único, con una clave que determina qué permutación se debe realizar.

> Un "bloque" se refiere simplemente a una cantidad fija de bits o bytes, que pueden representar cualquier tipo de datos. AES procesa un bloque y genera otro bloque. Hablaremos específicamente de la variante de AES que funciona con bloques de 128 bits (16 bytes) y una clave de 128 bits, conocida como AES-128.

Usando la misma clave, la permutación puede realizarse a la inversa, mapeando el bloque de salida nuevamente al bloque de entrada original. Es importante que exista una correspondencia uno a uno entre los bloques de entrada y salida, de otra manera no podríamos confiar en el texto cifrado para descifrarlo nuevamente al mismo texto simple con el que comenzamos.

¿Cuál es el término matemático para una correspondencia uno a uno?

In [ ]:
# Una biyección (bijection) es una correspondencia entre dos conjuntos que asigna a cada elemento de un conjunto un único elemento del otro conjunto

## Resisting Bruteforce

Si un cifrado de bloques es seguro, no debería haber forma de que un atacante distinga la salida de AES de una permutación aleatoria de bits. Además, no debería haber una mejor forma de deshacer la permutación que simplemente forzando bruscamente cada clave posible. Es por eso que los académicos consideran que un cifrado está teóricamente "roto" si pueden encontrar un ataque que requiera menos pasos para realizarse que forzar bruscamente la clave, incluso si ese ataque es prácticamente inviable.

> ¿Qué tan difícil es forzar un espacio de claves de 128 bits? [Alguien calculó](https://crypto.stackexchange.com/questions/48667/how-long-would-it-take-to-brute-force-an-aes-128-key/48669#48669) que si se utilizara la potencia de toda la red de minería de Bitcoin contra una clave AES-128, se necesitarían cien veces la edad del universo para descifrar la clave.

Resulta que existe un [ataque](https://en.wikipedia.org/wiki/Biclique_attack) contra AES que es mejor que el de fuerza bruta, pero solo un poco: reduce el nivel de seguridad de AES-128 a 126,1 bits y no se ha mejorado en más de 8 años. Dado el gran "margen de seguridad" que proporcionan los 128 bits y la falta de mejoras a pesar de un estudio exhaustivo, no se considera un riesgo creíble para la seguridad de AES. Pero sí, en un sentido muy estricto, "rompe" AES.

Finalmente, si bien las computadoras cuánticas tienen el potencial de romper por completo los criptosistemas de clave pública populares como RSA a través del [algoritmo de Shor](https://es.wikipedia.org/wiki/Algoritmo_de_Shor), se cree que solo reducen a la mitad el nivel de seguridad de los criptosistemas simétricos a través del [algoritmo de Grover](https://es.wikipedia.org/wiki/Algoritmo_de_Grover). Esta es una de las razones por las que la gente recomienda usar AES-256, a pesar de que tiene un rendimiento menor, ya que aún proporcionaría una seguridad de 128 bits muy adecuada en un futuro cuántico.

¿Cuál es el nombre del mejor ataque de clave única contra AES?

In [ ]:
# El ataque biclique es una variante del método de criptoanálisis de encuentro en el medio (MITM).
# Utiliza una estructura biclique para extender el número de rondas que pueden ser atacadas por el ataque MITM.

## Structure of AES

Para lograr una permutación con clave que no sea factible de invertir sin la clave, AES aplica una gran cantidad de operaciones de mezcla ad hoc en la entrada. Esto contrasta marcadamente con los criptosistemas de clave pública como RSA, que se basan en elegantes problemas matemáticos individuales. AES es mucho menos elegante, pero es muy rápido.

En un nivel alto, AES-128 comienza con un "programa de clave" y luego ejecuta 10 rondas sobre un estado. El estado inicial es simplemente el bloque de texto simple que queremos cifrar, representado como una matriz de bytes de 4x4. A lo largo de las 10 rondas, el estado se modifica repetidamente mediante una serie de transformaciones invertibles.

> Cada paso de transformación tiene un propósito definido basado en las propiedades teóricas de los cifrados seguros establecidos por Claude Shannon en la década de 1940. Analizaremos cada uno de ellos más de cerca en los siguientes desafíos.

A continuación se muestra una descripción general de las fases del cifrado AES:

1. **KeyExpansion o Key Schedule**

A partir de la clave de 128 bits, se derivan 11 "claves de ronda" independientes de 128 bits: una para utilizar en cada paso AddRoundKey.

2. **Adición de clave inicial**

AddRoundKey: los bytes de la clave de la primera ronda se combinan con los bytes del estado.

3. **Ronda**: esta fase se repite 10 veces, para 9 rondas principales más una "ronda final"

  a) SubBytes: cada byte del estado se sustituye por un byte diferente según una tabla de búsqueda ("S-box").

  b) ShiftRows: las últimas tres filas de la matriz de estado se transponen (se desplazan una, dos o tres columnas).

  c) MixColumns: se realiza la multiplicación de matrices en las columnas del estado, combinando los cuatro bytes de cada columna. Esto se omite en la ronda final.

  d) AddRoundKey: los bytes de la clave de la ronda actual se combinan con los bytes del estado.

Se incluye una función `bytes2matrix` para convertir nuestro bloque de texto simple inicial en una matriz de estado. Escriba una función `matrix2bytes` para convertir esa matriz nuevamente en bytes y envíe el texto simple resultante como bandera.

Archivos del desafío:
- [matrix.py](https://cryptohack.org/static/challenges/matrix_e1b463dddbee6d17959618cf370ff1a5.py)

Recursos:
- [YouTube: Explicación del cifrado AES Rijndael como una animación Flash](https://www.youtube.com/watch?v=gP4PqVGudtg)

In [ ]:
def bytes2matrix(text):
  """ Converts a 16-byte array into a 4x4 matrix.  """
  return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
  """ Converts a 4x4 matrix into a 16-byte array.  """
  return bytes([i for row in matrix for i in row])

matrix = [
  [99, 114, 121, 112],
  [116, 111, 123, 105],
  [110, 109, 97, 116],
  [114, 105, 120, 125],
]

matrix2bytes(matrix)

b'crypto{inmatrix}'

## Round Keys

Por ahora, vamos a pasar por alto los detalles más finos de la fase **KeyExpansion**. El punto principal es que toma nuestra clave de 16 bytes y produce 11 matrices de 4x4 llamadas "claves redondas" derivadas de nuestra clave inicial. Estas claves redondas permiten que AES aproveche al máximo la clave única que proporcionamos.

La fase **inicial de adición de claves**, que es la siguiente, consta de un único paso *AddRoundKey*. El paso *AddRoundKey* es sencillo: realiza una operación XOR entre el estado actual y la clave de ronda actual.

*AddRoundKey* también se produce como el paso final de cada ronda. *AddRoundKey* es lo que hace que AES sea una "permutación con clave" en lugar de solo una permutación. Es la única parte de AES donde la clave se mezcla con el estado, pero es crucial para determinar la permutación que ocurre.

Como has visto en desafíos anteriores, XOR es una operación que se puede invertir fácilmente si conoces la clave, pero es difícil de deshacer si no la conoces. Ahora imagina intentar recuperar texto sin formato que se ha XOR con 11 claves diferentes y que se ha mezclado mucho entre cada operación XOR con una serie de cifras de sustitución y transposición. ¡Eso es más o menos lo que hace AES! Y veremos cuán efectiva es la mezcla en los próximos desafíos.

Completa la función `add_round_key`, luego usa la función `matrix2bytes` para obtener tu próxima bandera.

Archivos del desafío:
* [add_round_key.py](https://cryptohack.org/static/challenges/add_round_key_b67b9a529ae739156107a74b14adde98.py)



In [ ]:
state = [
    [206, 243, 61, 34],
    [171, 11, 93, 31],
    [16, 200, 91, 108],
    [150, 3, 194, 51],
]

round_key = [
    [173, 129, 68, 82],
    [223, 100, 38, 109],
    [32, 189, 53, 8],
    [253, 48, 187, 78],
]

def matrix2bytes(matrix):
    """ Converts a 4x4 matrix into a 16-byte array.  """
    return bytes([i for row in matrix for i in row])

def add_round_key(s, k):
    return matrix2bytes([[s[i][j] ^ k[i][j] for j in range(len(s[i]))] for i in range(len(s))])

add_round_key(state, round_key)

## Confusion through Substitution

El primer paso de cada ronda de AES es SubBytes. Esto implica tomar cada byte de la matriz de estado y sustituirlo por un byte diferente en una tabla de búsqueda preestablecida de 16x16. La tabla de búsqueda se denomina "caja de sustitución" o "caja S" para abreviar, y puede resultar confusa a primera vista. Vamos a desglosarla.

En 1945, el matemático estadounidense Claude Shannon publicó un artículo innovador sobre la teoría de la información, en el que identificaba la "confusión" como una propiedad esencial de un cifrado seguro. "Confusión" significa que la relación entre el texto cifrado y la clave debe ser lo más compleja posible. Dado solo un texto cifrado, no debería haber forma de saber nada sobre la clave.

Si un cifrado tiene poca confusión, es posible expresar una relación entre el texto cifrado, la clave y el texto simple como una función lineal. Por ejemplo, en un cifrado César, `texto cifrado = texto simple + clave`. Esa es una relación obvia, que es fácil de revertir. Las transformaciones lineales más complicadas se pueden resolver utilizando técnicas como la eliminación gaussiana. Incluso los polinomios de bajo grado, por ejemplo, una ecuación como `x^4 + 51x^3 + x`, se pueden resolver de manera eficiente [utilizando métodos algebraicos](https://math.stackexchange.com/a/1078515). Sin embargo, cuanto mayor sea el grado de un polinomio, generalmente más difícil será resolverlo: solo se puede aproximar mediante una cantidad cada vez mayor de funciones lineales.

El objetivo principal de la S-box es transformar la entrada de una manera que sea resistente a ser aproximada por funciones lineales. Las S-boxes apuntan a una alta no linealidad y, si bien la de AES no es perfecta, se acerca bastante. La búsqueda rápida en una S-box es un atajo para realizar una función muy no lineal en los bytes de entrada. Esta función implica tomar la inversa modular en el [campo de Galois 2**8](https://www.samiam.org/galois.html) y luego aplicar una transformación afín que se ha modificado para generar la máxima confusión. La forma más simple de expresar la función es mediante el siguiente polinomio de alto grado.

Para crear la S-box, se calculó la función sobre todos los valores de entrada desde 0x00 hasta 0xff y las salidas se colocaron en la tabla de búsqueda.

Implementar sub_bytes, enviar la matriz de estado a través de la S-box inversa y luego convertirla a bytes para obtener el indicador.

Archivos de desafío:
- [sbox.py](https://cryptohack.org/static/challenges/sbox_8fc04ffb95faf5a5e6959195d5e2d94e.py)


In [ ]:
s_box = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

inv_s_box = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

state = [
    [251, 64, 182, 81],
    [146, 168, 33, 80],
    [199, 159, 195, 24],
    [64, 80, 182, 255],
]

def inv_sub_bytes(s, sbox=inv_s_box):
    return [[sbox[byte] for byte in row] for row in s]

def matrix2bytes(matrix):
    return ''.join(chr(byte) for row in matrix for byte in row)

matrix2bytes(inv_sub_bytes(state, sbox=inv_s_box))

'crypto{l1n34rly}'

## Diffusion through Permutation

Hemos visto cómo la sustitución de la caja S genera confusión. La otra propiedad crucial descrita por Shannon es la "difusión". Se relaciona con la forma en que cada parte de la entrada de un cifrado debe propagarse a cada parte de la salida.

La sustitución por sí sola crea no linealidad, pero no la distribuye por todo el estado. Sin la difusión, el mismo byte en la misma posición recibiría las mismas transformaciones aplicadas en cada ronda. Esto permitiría a los criptoanalistas atacar cada posición de byte en la matriz de estado por separado. Necesitamos alternar las sustituciones mezclando el estado (de manera invertible) de modo que las sustituciones aplicadas en un byte influyan en todos los demás bytes del estado. Cada entrada en la siguiente caja S se convierte entonces en una función de múltiples bytes, lo que significa que con cada ronda la complejidad algebraica del sistema aumenta enormemente.

> Una cantidad ideal de difusión hace que un cambio de un bit en el texto simple dé lugar a un cambio en la mitad estadística de los bits del texto cifrado. Este resultado deseable se denomina efecto avalancha.

Los pasos ShiftRows y MixColumns se combinan para lograr esto. Trabajan juntos para garantizar que cada byte afecte a todos los demás bytes del estado en solo dos rondas.

ShiftRows es la transformación más simple en AES. Mantiene la primera fila de la matriz de estado igual. La segunda fila se desplaza una columna hacia la izquierda, dando vueltas. La tercera fila se desplaza dos columnas, la cuarta fila tres. Wikipedia lo expresa muy bien: "la importancia de este paso es evitar que las columnas se cifren de forma independiente, en cuyo caso AES degenera en cuatro cifrados de bloque independientes".

> El diagrama (y la especificación AES) muestran la operación ShiftRows en notación de columna principal. Sin embargo, el código de muestra a continuación utiliza la notación de fila principal para la matriz de estado, ya que es más natural en Python. Siempre que se utilice la misma notación cada vez que se accede a la matriz, el resultado final es idéntico. Debido a los patrones de acceso y al comportamiento de la memoria caché, el uso de un tipo de notación puede generar un mejor rendimiento.

MixColumns es más complejo. Realiza la multiplicación de matrices en el campo de Galois de Rijndael entre las columnas de la matriz de estado y una matriz preestablecida. Por lo tanto, cada byte de cada columna afecta a todos los bytes de la columna resultante. Los detalles de implementación son matizados; [esta página](https://www.samiam.org/mix-column.html) y [Wikipedia](https://en.wikipedia.org/wiki/Rijndael_MixColumns) hacen un buen trabajo al cubrirlos.

Hemos proporcionado el código para ejecutar MixColumns y la operación ShiftRows hacia adelante. Después de implementar `inv_shift_rows`, tome el estado, ejecute `inv_mix_columns` en él, luego `inv_shift_rows`, conviértalo a bytes y tendrá su bandera.

Archivos de desafío:
- [diffusion.py](https://cryptohack.org/static/challenges/diffusion_ee6215282094b4ae8cd1b20697477712.py)



In [ ]:
def shift_rows(s):
  s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
  s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
  s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]

def inv_shift_rows(s):
  s[1][1], s[2][1], s[3][1], s[0][1] = s[0][1], s[1][1], s[2][1], s[3][1]
  s[2][2], s[3][2], s[0][2], s[1][2] = s[0][2], s[1][2], s[2][2], s[3][2]
  s[3][3], s[0][3], s[1][3], s[2][3] = s[0][3], s[1][3], s[2][3], s[3][3]

# learned from http://cs.ucsb.edu/~koc/cs178/projects/JT/aes.c
xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)

def mix_single_column(a):
    # see Sec 4.1.2 in The Design of Rijndael
    t = a[0] ^ a[1] ^ a[2] ^ a[3]
    u = a[0]
    a[0] ^= t ^ xtime(a[0] ^ a[1])
    a[1] ^= t ^ xtime(a[1] ^ a[2])
    a[2] ^= t ^ xtime(a[2] ^ a[3])
    a[3] ^= t ^ xtime(a[3] ^ u)

def mix_columns(s):
    for i in range(4):
        mix_single_column(s[i])

def inv_mix_columns(s):
    # see Sec 4.1.3 in The Design of Rijndael
    for i in range(4):
        u = xtime(xtime(s[i][0] ^ s[i][2]))
        v = xtime(xtime(s[i][1] ^ s[i][3]))
        s[i][0] ^= u
        s[i][1] ^= v
        s[i][2] ^= u
        s[i][3] ^= v

    mix_columns(s)

state = [
    [108, 106, 71, 86],
    [96, 62, 38, 72],
    [42, 184, 92, 209],
    [94, 79, 8, 54],
]

inv_mix_columns(state)

inv_shift_rows(state)

def matrix2bytes(matrix):
    return ''.join(chr(byte) for row in matrix for byte in row)

matrix2bytes(state)

## Bringing It All Together

Aparte de la fase **KeyExpansion**, hemos esbozado todos los componentes de AES. Hemos mostrado cómo *SubBytes* genera confusión y *ShiftRows* y *MixColumns* generan difusión, y cómo estas dos propiedades trabajan juntas para hacer circular repetidamente transformaciones no lineales por el estado. Por último, AddRoundKey introduce la clave en esta red de sustitución-permutación, lo que convierte el cifrado en una permutación con clave.

El descifrado implica realizar los pasos descritos en el desafío "Estructura de AES" en sentido inverso, aplicando las operaciones inversas. Tenga en cuenta que KeyExpansion aún debe ejecutarse primero y las claves de redondeo se utilizarán en orden inverso. AddRoundKey y su inversa son idénticas, ya que XOR tiene la propiedad de autoinversión.

Hemos proporcionado el código de expansión de clave y el texto cifrado que se ha cifrado correctamente con AES-128. Copia todos los bloques de construcción que has codificado hasta ahora y completa la función de `descifrado` que implementa los pasos que se muestran en el diagrama. El texto sin formato descifrado es el indicador.

Sí, puedes hacer trampa en este desafío, pero ¿dónde está la diversión en eso?

El código utilizado en estos ejercicios se ha tomado de la implementación súper simple de AES en Python de Bo Zhu, por lo que reproducimos la licencia aquí.

Archivos del desafío:
- [aes_decrypt.py](https://cryptohack.org/static/challenges/aes_decrypt_f491744105801ec03d6a6f7a0e7f8101.py)
- [LICENCIA](https://cryptohack.org/static/challenges/LICENSE_651b0602addd2b4bfa5d69ad7fca2dd5)

Recursos:
- [Creando tu propia criptografía: Todo lo que necesitas para crear AES desde cero](https://github.com/francisrstokes/githublog/blob/main/2022/6/15/rolling-your-own-crypto-aes.md)

In [ ]:
## CODE

# Symmetric Starter

## Modes of Operation Starter

El conjunto anterior de desafíos mostró cómo AES realiza una permutación con clave en un bloque de datos. En la práctica, necesitamos cifrar mensajes mucho más largos que un solo bloque. Un modo de operación describe cómo usar un cifrado como AES en mensajes más largos.

Todos los modos tienen serias debilidades cuando se usan incorrectamente. Los desafíos de esta categoría lo llevan a una sección diferente del sitio web donde puede interactuar con las API y explotar esas debilidades. ¡Familiarícese con la interfaz y úsela para obtener su próxima bandera!

Juegue en [https://aes.cryptohack.org/block_cipher_starter](https://aes.cryptohack.org/block_cipher_starter)

In [ ]:
import requests

URL = 'https://aes.cryptohack.org//block_cipher_starter/'

response = requests.get(URL + 'encrypt_flag/')

if response.status_code == 200:
  encrypt_flag = response.json()['ciphertext']

response = requests.get(URL + 'decrypt/' + encrypt_flag + '/')

if response.status_code == 200:
  decrypt_flag = response.json()['plaintext']
  flag_str = bytes.fromhex(decrypt_flag)

flag_str

## Passwords as Keys

Es esencial que las claves en los algoritmos de clave simétrica sean bytes aleatorios, en lugar de contraseñas u otros datos predecibles. Los bytes aleatorios se deben generar utilizando un generador de números pseudoaleatorios criptográficamente seguro (CSPRNG). Si las claves son predecibles de alguna manera, entonces el nivel de seguridad del cifrado se reduce y es posible que un atacante que obtenga acceso al texto cifrado pueda descifrarlo.

El hecho de que una clave parezca estar formada por bytes aleatorios no significa necesariamente que lo esté. En este caso, la clave se ha derivado de una contraseña simple utilizando una función hash, lo que hace que el texto cifrado sea descifrable.

Para este desafío, puede crear secuencias de comandos para sus solicitudes HTTP a los puntos finales o, alternativamente, atacar el texto cifrado sin conexión. ¡Buena suerte!

Juega en [https://aes.cryptohack.org/passwords_as_keys](https://aes.cryptohack.org/passwords_as_keys)

In [ ]:
import requests
import hashlib
from multiprocessing import Pool

URL_WORDS = "https://gist.githubusercontent.com/wchargin/8927565/raw/d9783627c731268fb2935a731a618aa8e95cf465/words"
URL = "https://aes.cryptohack.org/passwords_as_keys/"

response = requests.get(URL_WORDS)

if response.status_code == 200:
  words = [w.strip() for w in response.text.splitlines()]

response = requests.get(URL + 'encrypt_flag/')

if response.status_code == 200:
  flag = response.json()['ciphertext']

def check(word):
  response = requests.get(URL + 'decrypt/' + flag + "/" + hashlib.md5(word.encode()).hexdigest() + '/')

  if response.status_code == 200:
    flag_decrypt = bytes.fromhex(response.json()['plaintext'])
    if(flag_decrypt.startswith(b"crypto")):
      return flag_decrypt
  return None

with Pool() as pool:
  for result in pool.imap(check, words):
    if result:
      print(f"{result}")

# Block Ciphers 1

## ECB CBC WTF

Aquí puede cifrar en CBC pero solo descifrar en ECB. Eso no debería ser una debilidad porque son modos diferentes... ¿verdad?

Play at [https://aes.cryptohack.org/ecbcbcwtf](https://aes.cryptohack.org/ecbcbcwtf)

In [ ]:
import requests

URL = 'https://aes.cryptohack.org/ecbcbcwtf/'

response = requests.get(URL + 'encrypt_flag/')

if response.status_code == 200:
  encrypt_flag = response.json()['ciphertext']

SIZE = 32

iv = encrypt_flag[0:SIZE]
c_0 = encrypt_flag[SIZE:SIZE*2]
c_1 = encrypt_flag[SIZE*2:SIZE*3]

response = requests.get(URL + 'decrypt/' + str(c_0) + '/')

if response.status_code == 200:
  d_0 = response.json()['plaintext']

response = requests.get(URL + 'decrypt/' + str(c_1) + '/')

if response.status_code == 200:
  d_1 = response.json()['plaintext']

for i in range(32):
  p_0 = hex(int(d_0, 16) ^ int(iv, 16))
  p_1 = hex(int(d_1, 16) ^ int(c_0, 16))

bytes.fromhex(p_0[2:]+p_1[2:])